In [1]:
import random
import json
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
from keras.models import load_model
import numpy as np
from abc import ABC
from flask import Flask, request, jsonify
from tensorflow.keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import requests
from bs4 import BeautifulSoup

In [79]:
# check if punkt tokenizer is available
try:
    nltk.data.find("tokenizers/punkt")
except LookupError:
    nltk.download("punkt")

# check if wordnet corpora is available
try:
    nltk.data.find("corpora/wordnet")
except LookupError:
    nltk.download("wordnet")


In [80]:
class Seq2SeqChatbot():
    intentss=None
    vocabulary=None
    classes=None
    model=None
    def __init__(self,name, intents_path, model_path,vocabulary_path,classes_path):
        self.name = name
        self.status = False
        self.intentss = json.loads(open(intents_path).read())
        self.vocabulary = pickle.load(open(vocabulary_path, "rb"))
        self.classes = pickle.load(open(classes_path, "rb"))
        self.model_name = model_path
        # options for language detection models
        self.lang_detect_model = ["googletrans", "fasttext"]
        # initialize Lemmatizer instance
        self.lemmatizer = WordNetLemmatizer()
    def load(self):
        self.model = load_model(self.model_name)
        self.status = True
        return True  



    # preprocess user input text
    def prepocess_text(self,text):
    # tokenize the text
        words = nltk.word_tokenize(text)
    # lemmatize each word
        words = [self.lemmatizer.lemmatize(word.lower()) for word in words]
        return words


# return bag of words array: 0 or 1 for each word in vocab present in sentence
    def bag_of_words(self,text, vocab, show_details=True):
    # preprocess text
        sentence_words = self.prepocess_text(text)
    # bag of words - vocabulary matrix - matrix of N words
        bag = [0] * len(vocab)
        for s in sentence_words:
            for i, w in enumerate(vocab):
                if w == s:
                # assign 1 if the word is in vocabulary
                    bag[i] = 1
                    if show_details:
                        print("found in bag: %s" % w)
        return np.array(bag)


# predict corresponding intent for input text
    def predict_intent(self,sentence, model):
        print("hello")
        p = self.bag_of_words(sentence, self.vocabulary, show_details=False)
        # filter out predictions by threshold
        res = model.predict(np.array([p]))[0]
        ERROR_THRESHOLD = 0.25
        results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
        # sort by probability score
        results.sort(key=lambda x: x[1], reverse=True)
        return_list = []
        for r in results:
            return_list.append({"intent": self.classes[r[0]], "probability": str(r[1])})
        
        return return_list


# generate output response based on identified intent of input
    def generate_response(self,ints, intents_json):
        tag = ints[0]["intent"]
        result="Sorry , we don't have this information , ask another question please ."
        list_of_intents = intents_json["intents"]
        for i in list_of_intents:
            if i["tag"] == tag:
            # select a response at random from available responses
                result = random.choice(i["responses"])
                break
        return result


# generate chatbot output for user input message
    def chatbot_response(self,msg):
        print(msg)
        ints = self.predict_intent(msg, self.model)
        res = self.generate_response(ints, self.intentss)
        return res
    def get_status(self):
        return self.status
    

In [81]:
class Gestion:
    DataBase = 'Chatbots.json'
    bots_objects = {}

    def __init__(self):
        with open(self.DataBase, 'r') as myfile:
            data=myfile.read()
        bots = json.loads(data)["chatbots"]
        
        
        for bot in bots:
            self.bots_objects[bot["intitule"]] = Seq2SeqChatbot(bot["intitule"],bot["json"],bot["modelName"],bot["vocabulary"],bot["classes"])
           

                
    def load_bot(self, name):
        if self.bots_objects[name].get_status():
            return True
        else:
            return self.bots_objects[name].load()

    def get_answer(self, name, query):
        print(query)
        if self.bots_objects[name].get_status():
            return self.bots_objects[name].chatbot_response(query)
        else:
            return 'please wait'

    #Creer une liste des chatbots pour l'utilisateur
    def list(self):
        lbot = []
        for key in self.bots_objects:
            lbot.append({
                "name": key,
                "uri": "https://picsum.photos/200/300",
                "color": "red",
                "describtion": "Ur chatbot buddy"
            })
        return lbot

    def add_proposition(self, app, name, question, response):
        pass

In [82]:
# Creer l'instance du serveur flask
import json
app = Flask('__main__')

#Creer l'instance du gestionnaire
gst = Gestion()

In [83]:
@app.route('/ask/<name>', methods=['GET'])
def hello_world(name):
    answer = gst.get_answer(name, str(request.args.get('query')))
    return jsonify(ans=answer)

In [84]:
@app.route('/load/<name>', methods=['GET'])
def load_bot(name):
    gst.load_bot(name)
    return jsonify(ans=True)

In [85]:
@app.route('/list', methods=['Get'])
def list_of_bots():
    lbot = gst.list()
    return jsonify(lbot)

@app.route('/propose/<name>', methods=['Post'])
def propose(name):
    gst.add_proposition(app, name, request.args.get('question'), request.args.get('response'))
    return jsonify(ans=True)

In [ ]:
app.run(host='192.168.0.188',port='19000')

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.0.188:19000/ (Press CTRL+C to quit)
192.168.0.188 - - [04/Jul/2021 18:42:20] "GET / HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 18:42:30] "GET //list HTTP/1.1" 200 -
192.168.0.188 - - [04/Jul/2021 18:42:30] "POST /logs HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 18:45:26] "GET / HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 18:45:53] "GET //list HTTP/1.1" 200 -
192.168.0.188 - - [04/Jul/2021 18:45:53] "POST /logs HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 18:45:54] "POST /logs HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 18:45:56] "POST /logs HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 18:46:01] "GET //load/where%20to%20go HTTP/1.1" 200 -
192.168.0.188 - - [04/Jul/2021 18:46:28] "GET //load/where%20to%20go HTTP/1.1" 200 -
192.168.0.188 - - [04/Jul/2021 18:46:32] "GET //load/hello HTTP/1.1" 200 -
192.168.0.188 - - [04/Jul/2021 18:46:39] "GET / HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 18:46:47] "GET //list HTTP/1.1" 200 -
192.168.0.188 -

hello
hello
hello


192.168.0.188 - - [04/Jul/2021 19:13:54] "GET //ask/where%20to%20go?query=hello HTTP/1.1" 200 -
192.168.0.188 - - [04/Jul/2021 19:17:18] "GET //load/where%20to%20go HTTP/1.1" 200 -


Hhhh
Hhhh
hello


...

192.168.0.188 - - [04/Jul/2021 19:19:15] "GET //load/where%20to%20go HTTP/1.1" 200 -


whats up
whats up
hello


192.168.0.188 - - [04/Jul/2021 19:19:23] "GET //ask/where%20to%20go?query=whats%20up HTTP/1.1" 200 -
192.168.0.188 - - [04/Jul/2021 19:29:11] "POST /logs HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 19:32:52] "POST /logs HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 19:32:53] "POST /logs HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 19:33:26] "GET //list HTTP/1.1" 200 -
192.168.0.188 - - [04/Jul/2021 19:33:27] "GET //list HTTP/1.1" 200 -
192.168.0.188 - - [04/Jul/2021 19:33:28] "POST /logs HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 19:33:28] "POST /logs HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 19:33:28] "GET //list HTTP/1.1" 200 -
192.168.0.188 - - [04/Jul/2021 19:33:29] "POST /logs HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 19:33:57] "GET / HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 19:34:06] "GET //list HTTP/1.1" 200 -
192.168.0.188 - - [04/Jul/2021 19:34:06] "POST /logs HTTP/1.1" 404 -
192.168.0.188 - - [04/Jul/2021 19:34:07] "POST /logs HTTP/1.1" 404 -
192.168

hello
hello
hello


192.168.0.188 - - [04/Jul/2021 19:37:51] "GET //ask/hello?query=hello HTTP/1.1" 200 -
